In [33]:
import spotipy
import math
import pandas as pd
import config

In [34]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(config.client_id, config.client_secret)

In [35]:
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [36]:
#id = '6XyY86QOPPrYVGvF9ch6wz' #Linkin Park
id = '51Blml2LZPmy7TTiAg47vQ' #U2

In [37]:
albums_list = spotify.artist_albums(artist_id=id, limit=50)
albums_list

{'href': 'https://api.spotify.com/v1/artists/51Blml2LZPmy7TTiAg47vQ/albums?offset=0&limit=50&include_groups=album,single,compilation,appears_on',
 'items': [{'album_group': 'album',
   'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/51Blml2LZPmy7TTiAg47vQ'},
     'href': 'https://api.spotify.com/v1/artists/51Blml2LZPmy7TTiAg47vQ',
     'id': '51Blml2LZPmy7TTiAg47vQ',
     'name': 'U2',
     'type': 'artist',
     'uri': 'spotify:artist:51Blml2LZPmy7TTiAg47vQ'}],
   'available_markets': ['JP'],
   'external_urls': {'spotify': 'https://open.spotify.com/album/3TbG4ZvwHmzkU9JGltkxnf'},
   'href': 'https://api.spotify.com/v1/albums/3TbG4ZvwHmzkU9JGltkxnf',
   'id': '3TbG4ZvwHmzkU9JGltkxnf',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/cf6df046315e3f28af7f5ffb569343c3b66d87de',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/19e628387ecf15a7e0e607c7338aa39220fca479',
     'width': 300},
  

In [6]:
albums = filter(lambda a: a['album_type'] != 'single', albums_list['items'])

In [7]:
for album in albums:
    print(album['name'], album['id'], album['release_date'], album['album_type'], album['type'])

Songs Of Experience (Deluxe Edition) 3TbG4ZvwHmzkU9JGltkxnf 2017-12-01 album album
Songs Of Experience (Deluxe Edition) 6S9YaGXnmRe8tWJ0e457HP 2017-12-01 album album
Songs Of Innocence 7DY6v8qiyQcsTCqUWjH7pS 2014-01-01 album album
Songs Of Innocence 45Ec0wYJ7npo6lXH4GWmxz 2014-01-01 album album
Songs Of Innocence (Deluxe) 6wQXpkptAIZPzNf5H0SA10 2014-01-01 album album
Songs Of Innocence (Deluxe) 5tWSTNTqzqZhKo7FCyLwPk 2014-01-01 album album
Songs Of Innocence (Deluxe) 7iOtAFn7s1kzvLYz7nsBBV 2014-01-01 album album
No Line On The Horizon 2cVfcuckyfRP6KVAHsRPHC 2009-01-01 album album
No Line On The Horizon 5yqMA7IvahYJc53ewRAMnP 2009-01-01 album album
How To Dismantle An Atomic Bomb 5PQPur1PEZFDkI0AXbxFlB 2004-01-01 album album
How To Dismantle An Atomic Bomb 2nPJlUlcyQ24e1VdayD6TT 2004-01-01 album album
All That You Can't Leave Behind 7a5U0GPoAvT3gvEY66FRuN 2000-01-01 album album
All That You Can't Leave Behind 7gskILm9UyDvFlmmAoqn2g 2000-01-01 album album
Pop 5mojJwWgWNJcY3odUGgQc3 1997-

In [8]:
#album_ids = ['5Eevxp2BCbWq25ZdiXRwYd', '3XB2yloP7l00tEUmaODtVi', '4XHIjbhjRmqWlosjj5rqSI', 
#             '5uvXx5ZQswNRFCdHR521YZ', '2FJ7JZqCtWXd9MjuBQhCP4', '4Gfnly5CzMJQqkUFfoHaP3',
#            '2NbO8RRVTVEUjHaxUVdMDT', '6PFPjumGRpZnBzqnDci6qJ']
#album_names = ['Onemorelight', 'Huntingparty', 'Livingthings', 'Thousandsuns', 'Minutes', 
#              'Meteora', 'Reanimation', 'Hybrid']
album_ids = ['0E01zOqeK9o8xiAVWVDekj', '595N69tnPNf3f023L7ftWa', '2P2KGKsq7ItViExKB2TLzJ', 
             '4JfQDfKe4N5NCEyMP1mYT6', '4mULDK6YXrFXTfSwvwm4M3', '7hIoJcH4ObWasDFq78u1x9',
            '5n52kyQKeUZs5ObZJejLQd', '0IYjMBLA9PgtXyRPlLmTDE']
album_names = ['Boy', 'October', 'War', 'The Unforgettable Fire', 'The Joshua Tree', 
              'Rattle And Hum', 'Achtung Baby', 'Zooropa']

In [9]:
album_replace = {}
for id,name in zip(album_ids, album_names):
    album_replace[id] = name

In [10]:
features_required = ['energy', 'duration_ms', 'loudness', 'valence']

In [11]:
def get_track_features(songs, service):
    track_ids = [foo['id'] for foo in songs]
    chunk_size = 50
    num_chunks = int(math.ceil(len(track_ids) / float(chunk_size)))
    features_add = []
    for i in range(num_chunks):
        chunk_track_ids = track_ids[i*chunk_size:(i+1)*chunk_size]
        chunk_features = service.audio_features(tracks=chunk_track_ids)
        features_add.extend(chunk_features)
    
    features_df = pd.DataFrame(features_add)
    features_df = features_df[features_required]
    return features_df

In [20]:
songs_df_data = []
for album_id in album_ids:
    #get songs in the album
    songs = spotify.album_tracks(album_id=album_id)['items']
    features = get_track_features(songs, spotify)
    features = features.values.tolist()
    for (track, feature) in zip(songs, features):
        t = track
        this_row = [t['id'], t['name'], album_id]       
        this_row.extend(feature)
        songs_df_data.append(this_row)
    print( album_id)

0E01zOqeK9o8xiAVWVDekj
595N69tnPNf3f023L7ftWa
2P2KGKsq7ItViExKB2TLzJ
4JfQDfKe4N5NCEyMP1mYT6
4mULDK6YXrFXTfSwvwm4M3
7hIoJcH4ObWasDFq78u1x9
5n52kyQKeUZs5ObZJejLQd
0IYjMBLA9PgtXyRPlLmTDE


In [21]:
len(songs_df_data)

106

In [22]:
import pandas as pd
songs_df = pd.DataFrame(songs_df_data, columns=['id', 'name', 'album', 'energy', 
                                                'duration_ms', 'loudness','valence']).drop_duplicates(subset='id')

In [23]:
songs_df.replace(to_replace=album_replace, inplace=True)

#### Remove reanimation - only considering studio albums

In [24]:
songs_df = songs_df[songs_df.album != "Reanimation"]
songs_df['duration'] = songs_df['duration_ms']/60000.0
songs_df.drop(['duration_ms'], inplace=True, axis=1)

In [25]:
import pickle

In [26]:
pickle.dump(songs_df, open('U2-lp-data.p', 'wb'))

In [27]:
songs_df

,id,name,album,energy,loudness,valence,duration
0,1lWiFcyiL6aeGOUl5BRzIG,I Will Follow,Boy,0.8470,-10.206,0.6950,3.617117
1,6vWYMKYdAPGZbbEXTpmxAO,Twilight,Boy,0.7060,-10.720,0.4670,4.366667
2,6N8B5MUyIbQpsXTkUbvqRo,An Cat Dubh,Boy,0.5860,-10.119,0.4530,4.781550
3,4gq5pc5CZh8RWrpgW8Rwv6,Into The Heart,Boy,0.3520,-12.183,0.0393,3.461783
4,58Hinaks3XjZfxR0uaGRnJ,Out Of Control,Boy,0.8040,-10.258,0.4430,4.224883
5,1cZOePe1othCjiaapmyMOf,Stories For Boys,Boy,0.8290,-9.504,0.5250,3.042217
6,5Rv9nErNVgc1g4WjEVPzAT,The Ocean,Boy,0.2140,-17.855,0.1660,1.578217
7,21aneJ6TjUNZEMJM5G9Z39,A Day Without Me,Boy,0.7570,-8.213,0.7370,3.218883
8,6hcMboVh9lSfLqq37Gmpnt,Another Time Another Place,Boy,0.7380,-10.380,0.3940,4.555550
9,7ICH2LhdFnwHw3VS5Y4rWH,The Electric Co.,Boy,0.7460,-8.964,0.2930,4.792883
